# 纳斯达克增长追踪（免费且尽量稳定的数据源优先）

**数据源优先级（默认）：**
1. **FRED** — `NASDAQCOM`（纳斯达克综合指数收盘价，免费，建议配置 API Key）；
2. **Stooq** — 无需密钥；若指数不可取，使用 **ONEQ**（更贴近纳综指）或 **QQQ**（纳指100）作为**代理**；
3. **Yahoo Finance** — 免费但可能频控，仅作兜底。

> 说明：FRED 的 `NASDAQCOM` 提供“收盘价”时间序列，足以计算涨跌幅（无需 OHLC）；ONEQ 是 Fidelity 的纳综指 ETF（更贴近 ^IXIC），QQQ 为纳指100 ETF（非纳综指）。

In [1]:
# 安装/导入依赖（若已安装将跳过）
import sys, subprocess, os
def ensure(pkg_import_name, pip_name=None):
    pip_name = pip_name or pkg_import_name
    try:
        __import__(pkg_import_name)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name, "-q"])

for mod, pip in [
    ("pandas", "pandas"),
    ("numpy", "numpy"),
    ("matplotlib", "matplotlib"),
    ("pandas_datareader", "pandas-datareader"),
    ("yfinance", "yfinance"),
    ("dateutil", "python-dateutil"),
    ("requests", "requests"),
]:
    ensure(mod, pip)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yf
from datetime import datetime, timedelta
from dateutil import parser

plt.rcParams["figure.figsize"] = (10, 5)
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
/Users/xuzhaolin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently

In [2]:
# === 配置区（可按需修改）===
INDEX_SYMBOL = "^IXIC"                 # 目标：纳斯达克综合指数
PROXY_PRIORITY = ["ONEQ", "QQQ"]       # 代理优先顺序：ONEQ 更贴近纳综指，其次 QQQ（纳指100）
PREFERRED_PROVIDERS = ["fred", "stooq", "yahoo"]  # 数据源优先顺序
START_DATE = "1985-01-01"

# FRED：强烈建议设置免费 API Key（环境变量），无 Key 也可能可用，但不保证
FRED_API_KEY = os.getenv("FRED_API_KEY", "d24f3620963ee31100fcc57f287ea2f1")

In [3]:
def _series_from_df(df, price_col='Close'):
    if price_col not in df.columns:
        raise RuntimeError(f"源数据缺少列: {price_col}. 实际列: {list(df.columns)}")
    s = df[price_col].copy()
    s.index = pd.to_datetime(s.index).tz_localize(None)
    s = s.sort_index()
    s = s[~s.index.duplicated(keep='last')]
    return s.rename("close")

def load_from_fred(start=START_DATE):
    s = pdr.DataReader("NASDAQCOM", "fred", start=start).rename("close")
    if s.empty:
        raise RuntimeError("FRED 返回空数据。可能需要设置 FRED_API_KEY 或稍后重试。")
    df = s.to_frame()
    meta = {"provider": "fred", "symbol_used": "NASDAQCOM", "proxy": False}
    return df, meta

def load_from_stooq(symbol=INDEX_SYMBOL, start=START_DATE, proxy_priority=PROXY_PRIORITY):
    used_symbol = symbol
    if symbol.startswith("^"):
        last_error = None
        for proxy in proxy_priority:
            try:
                df = pdr.DataReader(proxy, "stooq", start=start)
                s = _series_from_df(df, price_col="Close")
                meta = {"provider": "stooq", "symbol_used": proxy, "proxy": True}
                return s.to_frame(), meta
            except Exception as e:
                last_error = e
        raise RuntimeError(f"Stooq 代理失败：{last_error}")
    else:
        df = pdr.DataReader(used_symbol, "stooq", start=start)
        s = _series_from_df(df, price_col="Close")
        meta = {"provider": "stooq", "symbol_used": used_symbol, "proxy": False}
        return s.to_frame(), meta

def load_from_yahoo(symbol=INDEX_SYMBOL, start=START_DATE):
    df = yf.download(symbol, start=start, progress=False, auto_adjust=False, threads=False)
    if df.empty:
        raise RuntimeError("Yahoo 返回空数据。")
    if "Adj Close" not in df.columns:
        raise RuntimeError("Yahoo 数据无 'Adj Close' 列。")
    s = df["Adj Close"].copy()
    s.index = pd.to_datetime(s.index).tz_localize(None)
    s = s.sort_index()
    s = s[~s.index.duplicated(keep='last')]
    meta = {"provider": "yahoo", "symbol_used": symbol, "proxy": False}
    return s.rename("close").to_frame(), meta

def load_series(preferred=PREFERRED_PROVIDERS):
    errors = {}
    for prov in preferred:
        try:
            if prov == "fred":
                return load_from_fred()
            elif prov == "stooq":
                return load_from_stooq()
            elif prov == "yahoo":
                return load_from_yahoo()
            else:
                errors[prov] = "未知数据源"
        except Exception as e:
            errors[prov] = str(e)
    raise RuntimeError(f"所有数据源均失败：{errors}")

data, meta = load_series()
latest_date = data.index[-1].date()
print("数据源：", meta)
print(f"数据截止（最后一个交易日）：{latest_date}")
print(f"运行时间：{pd.Timestamp.now()}")
data.tail()

数据源： {'provider': 'stooq', 'symbol_used': 'ONEQ', 'proxy': True}
数据截止（最后一个交易日）：2025-11-10
运行时间：2025-11-11 10:34:37.678362


,close
Date,
2025-11-04,91.960000
2025-11-05,92.440000
2025-11-06,90.660000
2025-11-07,90.450000
2025-11-10,92.560000


In [4]:
def ytd_return(df: pd.DataFrame):
    today = pd.Timestamp.today().tz_localize(None).normalize()
    year_start = pd.Timestamp(year=today.year, month=1, day=1)
    start_idx = df.index.searchsorted(year_start, side="left")
    if start_idx >= len(df):
        raise ValueError("今年还没有交易数据。")
    start_date = df.index[start_idx]
    end_date = df.index[-1]
    start_price = df.loc[start_date, "close"]
    end_price = df.loc[end_date, "close"]
    ret = end_price / start_price - 1.0
    return {
        "metric": "YTD",
        "start_date": start_date.date(),
        "end_date": end_date.date(),
        "start_price": float(start_price),
        "end_price": float(end_price),
        "abs_change": float(end_price - start_price),
        "pct_change": float(ret),
        "log_return": float(np.log(end_price / start_price)),
    }

def trailing_1y_return(df: pd.DataFrame):
    end_date = df.index[-1]
    candidate = end_date - pd.Timedelta(days=365)
    idx = df.index.searchsorted(candidate, side="right") - 1
    idx = max(0, idx)
    start_date = df.index[idx]
    start_price = df.loc[start_date, "close"]
    end_price = df.loc[end_date, "close"]
    ret = end_price / start_price - 1.0
    return {
        "metric": "Trailing_1Y",
        "start_date": start_date.date(),
        "end_date": end_date.date(),
        "start_price": float(start_price),
        "end_price": float(end_price),
        "abs_change": float(end_price - start_price),
        "pct_change": float(ret),
        "log_return": float(np.log(end_price / start_price)),
    }

summary_df = pd.DataFrame([ytd_return(data), trailing_1y_return(data)]).set_index("metric")
summary_df

,start_date,end_date,start_price,end_price,abs_change,pct_change,log_return
metric,,,,,,,
YTD,2025-01-02,2025-11-10,75.940000,92.560000,16.620000,0.218857,0.197914
Trailing_1Y,2024-11-08,2025-11-10,76.040000,92.560000,16.520000,0.217254,0.196598


In [5]:
def nearest_trading_date(df_index: pd.DatetimeIndex, target: pd.Timestamp, use_prev=True):
    if use_prev:
        idx = df_index.searchsorted(target, side="right") - 1
        idx = max(0, idx)
    else:
        idx = df_index.searchsorted(target, side="left")
        idx = min(idx, len(df_index)-1)
    return df_index[idx]

def compare_return(df: pd.DataFrame, start_date, end_date, plot=False):
    s_raw = pd.Timestamp(parser.parse(str(start_date)).date())
    e_raw = pd.Timestamp(parser.parse(str(end_date)).date())
    if s_raw > e_raw:
        s_raw, e_raw = e_raw, s_raw

    s_dt = nearest_trading_date(df.index, s_raw, use_prev=True)
    e_dt = nearest_trading_date(df.index, e_raw, use_prev=True)

    s_px = df.loc[s_dt, "close"]
    e_px = df.loc[e_dt, "close"]

    abs_change = float(e_px - s_px)
    pct_change = float(e_px / s_px - 1.0)
    log_ret = float(np.log(e_px / s_px))

    result = {
        "provider": meta.get("provider"),
        "symbol_used": meta.get("symbol_used"),
        "proxy_used": meta.get("proxy"),
        "start_date_input": str(start_date),
        "end_date_input": str(end_date),
        "start_date_trading": str(s_dt.date()),
        "end_date_trading": str(e_dt.date()),
        "start_price": float(s_px),
        "end_price": float(e_px),
        "abs_change": abs_change,
        "pct_change": pct_change,
        "log_return": log_ret,
    }

    if plot:
        plt.figure()
        df.loc[s_dt:e_dt, "close"].plot()
        plt.title(f"{meta.get('symbol_used')} {s_dt.date()} → {e_dt.date()}")
        plt.xlabel("Date"); plt.ylabel("Close")
        plt.show()
    return result

# 示例（可修改或注释）：
example = compare_return(data, "2024-01-01", pd.Timestamp.today().date(), plot=False)
pd.Series(example)

provider                   stooq
symbol_used                 ONEQ
proxy_used                  True
start_date_input      2024-01-01
end_date_input        2025-11-11
start_date_trading    2023-12-29
end_date_trading      2025-11-10
start_price            59.240000
end_price              92.560000
abs_change             33.320000
pct_change              0.562458
log_return              0.446260
dtype: object

In [6]:
# 交互：输入两个日期（YYYY-MM-DD），回车可跳过
try:
    s = input("输入开始日期 YYYY-MM-DD（直接回车跳过）：").strip()
    e = input("输入结束日期 YYYY-MM-DD（直接回车跳过）：").strip()
    if s and e:
        res = compare_return(data, s, e, plot=True)
        display(pd.Series(res))
    else:
        print("已跳过交互输入。")
except Exception as err:
    print("交互输入失败：", err)

KeyboardInterrupt: Interrupted by user

---
### 使用建议
- **优先 FRED**：免费、稳定，建议在系统环境变量中设置 `FRED_API_KEY`（注册免费）。  
- **ONEQ 代理优先**：若无法直接获取 ^IXIC，ONEQ 比 QQQ 更接近纳综指结构。  
- **严格对齐 ^IXIC**：若你有企业级/付费源（Tiingo/Polygon/Nasdaq Data Link 等），可将 `load_from_*` 中任意一个替换为对应 SDK/HTTP 调用。  
- **两日比较**：支持任意自然日输入，代码会自动对齐到最近的交易日（向下对齐）。